In [3]:
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from scipy.stats import multinomial
from collections import OrderedDict

In [4]:
np.random.seed(1)
mat = io.loadmat('tempNs4.mat')
model = {}
model['H'] = mat['H']
model['Wopt'] = mat['Wopt']
model['Fopt'] = mat['Fopt']
model['Ns'] = mat['Ns'][0,0] ## need to change nb of stream in channel realization, this case 1
# model['Ns'] = 2
model['Nt'] = mat['Nt'][0,0]
model['Nr'] = mat['Nr'][0,0]
model['Nc'] = mat['Nc'][0,0]
model['Nray'] = mat['Nray'][0,0]
model['realization'] = mat['realization'][0,0]
model['At'] = mat['At']
model['Ar'] = mat['Ar']
model['Nrf'] = 4
archi = 'FC'

In [49]:
np.diagflat([1,2,3,4])

array([[1, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 4]])

In [42]:
V_hat = np.zeros((model['Nt'],model['Ns'],model['realization']),dtype=complex)
U_hat = np.zeros((model['Nr'],model['Ns'],model['realization']),dtype=complex)
sigma_hat = np.zeros((model['Ns'],model['Ns'],model['realization']),dtype=complex)

In [51]:
for reali in range(model['realization']):
    if (np.linalg.matrix_rank(model['H'][:,:,reali])>=model['Ns']):
#         count = count + 1
        U, s, V = np.linalg.svd(model['H'][:,:,reali], full_matrices=True)
        V = V.conj().T
        V_hat[:,:,reali] = V[0:model['Nt'],0:model['Ns']]
        U_hat[:,:,reali] = U[0:model['Nr'],0:model['Ns']]
        sigma_hat[:,:,reali] = np.diagflat(s[:model['Ns']])

In [57]:
U_hat0 = U_hat[:,:,0]
V_hat0 = V_hat[:,:,0]
sigma_hat0 = sigma_hat[:,:,0]


In [58]:
print(V_hat0.shape)
print(U_hat0.shape)
print(sigma_hat0.shape)

(64, 4)
(16, 4)
(4, 4)


In [9]:
H0 = model['H'][:,:,0]

In [10]:
H0.shape

(16, 64)

In [60]:
F_rf = np.zeros((model['Nt'],model['Nrf']),dtype=complex)
W_rf = np.zeros((model['Nr'],model['Nrf']),dtype=complex)

In [61]:
W_rf.shape

(16, 4)

In [63]:
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [59]:
alpha = 0.0001
n_bit = 2
q = 2*np.pi/2**n_bit

In [311]:
F_rf_angle = np.zeros((model['Nt'],model['Nrf']))
W_rf_angle = np.zeros((model['Nr'],model['Nrf']))
F_rf = np.exp(1j*F_rf_angle)
W_rf = np.exp(1j*W_rf_angle)

In [310]:
F_rf_angle = np.zeros((model['Nt'],model['Nrf']))
W_rf_angle = np.zeros((model['Nr'],model['Nrf']))
F_rf = np.exp(1j*F_rf_angle)
W_rf = np.exp(1j*W_rf_angle)

for l in range(1):
    Frf_l = np.delete(F_rf, l, axis=1)
    Wrf_l = np.delete(W_rf, l, axis=1)
    fl = np.array([F_rf[:,[l]]])
    wl = np.array([W_rf[:,[l]]])
    Ql = U_hat0@np.linalg.inv(alpha*np.eye(model['Ns']) + sigma_hat0@V_hat0.conj().T@Frf_l@Wrf_l.conj().T@U_hat0)@sigma_hat0@V_hat0.conj().T

    vl = np.angle(fl)
    pl = np.angle(wl)
    for k in range(10):
        for i in range(model['Nt']):
            Ql_i = np.delete(Ql,i,axis=1)
            Qli = Ql[:,i].reshape(-1,1)
            v = np.delete(vl,i).reshape(-1,1)
            vect_Nt = Ql_i@np.exp(1j*v)
            vli_tilde = (np.angle(np.sum(np.exp(1j*pl)*vect_Nt)) - np.angle(np.sum(np.exp(1j*pl)*Qli)))%np.pi
            vl[i] = q * np.round(vli_tilde/q)
            print(np.angle(np.sum(np.exp(1j*pl)*vect_Nt)) )
            print(np.angle(np.sum(np.exp(1j*pl)*Qli)))
            print(vli_tilde)
            print(vl[i])
            print('------------')
            
        for j in range(model['Nr']):
            Ql_j = np.delete(Ql,j,axis=0)
            Qlj = Ql[j,:].reshape(-1,1)
            p = np.delete(wl,j).reshape(-1,1)
            vect_Nr = Ql_j.T@np.exp(1j*p)
            pli_tilde = (np.angle(np.sum(np.exp(1j*vl)*vect_Nr)) - np.angle(np.sum(np.exp(1j*vl)*Qlj)))%np.pi
            pl[j] = q * np.round(pli_tilde/q)
            
#         w = np.exp(1j*pl)
#         f = np.exp(1j*vl)
#         print(np.abs(w.conj().T@Ql@f))


2.499695741274459
-0.26554496494691965
2.7652407062213786
[3.14159265]
------------
2.606876927523038
-0.4209501551153304
3.0278270826383684
[3.14159265]
------------
2.7273502846263677
1.2865727059118923
1.4407775787144754
[1.57079633]
------------
2.665167486060715
-2.8800362972334934
2.403611129704416
[3.14159265]
------------
2.4647894241728032
-2.233461230552181
1.5566580011351911
[1.57079633]
------------
2.46867238908864
2.434517663967972
0.03415472512066797
[0.]
------------
2.4359068376903386
-0.9852246702513971
0.2795388543519426
[0.]
------------
2.2977764627276978
-2.385963332432727
1.5421471415706316
[1.57079633]
------------
2.4198294806411726
-0.24933856647317018
2.6691680471143426
[3.14159265]
------------
2.5646826197652017
2.04870426791821
0.5159783518469916
[0.]
------------
2.5580899072500167
1.7707041444114258
0.787385762838591
[1.57079633]
------------
2.468632459664723
-2.6681676559965504
1.9952074620714804
[1.57079633]
------------
2.452725181428529
-2.974647740

------------
-1.9033975918235275
2.0233625898239183
2.3564251255321405
[3.14159265]
------------
-1.2840880393106606
-1.9288726034531243
0.6447845641424637
[0.]
------------
-1.4175312220410863
2.004521213520551
2.861132871617949
[3.14159265]
------------
-0.925447570393765
-1.747969318261739
0.822521747867974
[1.57079633]
------------
-0.6481065118437132
0.03732450685308936
2.4561616348929904
[3.14159265]
------------
-0.5150702329933773
-2.1541209858931016
1.6390507528997245
[1.57079633]
------------
-0.9141059307801859
1.1111047215537004
1.1163820012559067
[1.57079633]
------------
-0.8822770484654726
-2.9571822879094682
2.0749052394439955
[1.57079633]
------------
-0.9979032235566331
2.114085999004483
0.02960343102867702
[0.]
------------
-1.4163485666970725
-2.321848671820738
0.9055001051236653
[1.57079633]
------------
-1.2860020540132826
1.619741794552802
0.23584880502370842
[0.]
------------
0.38599899881417865
1.9848565681074875
1.5427350842964844
[1.57079633]
------------
0.4

In [433]:
F_rf_angle = np.zeros((model['Nt'],model['Nrf']))
W_rf_angle = np.zeros((model['Nr'],model['Nrf']))
F_rf = np.exp(1j*F_rf_angle)
W_rf = np.exp(1j*W_rf_angle)
l=0
i=0
Frf_l = np.delete(F_rf, l, axis=1)
Wrf_l = np.delete(W_rf, l, axis=1)
fl = F_rf[:,[l]]
wl = W_rf[:,[l]]
Ql = U_hat0@np.linalg.inv(alpha*np.eye(model['Ns']) + sigma_hat0@V_hat0.conj().T@Frf_l@Wrf_l.conj().T@U_hat0)@sigma_hat0@V_hat0.conj().T

vl = np.angle(fl)
pl = np.angle(wl)

In [434]:
for i in range(1):
#     print(np.abs(np.exp(1j*pl).T@Ql@np.exp(1j*vl)))
# for i in range(5):
    Ql_i = np.delete(Ql,i,axis=1)
    q = Ql[:,[i]]
    v = np.delete(vl,i).reshape(-1,1)
#     vect_Nt = Ql_i@np.exp(1j*v)
    vli_tilde = ((np.angle(np.exp(1j*pl).T@Ql_i@np.exp(1j*v))) - np.angle(np.exp(1j*pl).T@q))%np.pi
    vl[i] = vli_tilde
#     vl[i] = q * np.round(vli_tilde/q)
    print(vl)
    

[[2.76524071]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]


In [478]:
a = np.exp(1j*np.array([[0,1]]))
b = np.array([[1+1j*2,3+1j*4],[5+1j*6,7+1j*8]])
c = np.exp(1j*np.array([[2],[3]]))

In [481]:
np.abs(a@b@c)

array([[21.70675643]])

In [487]:
ang1 = np.angle(a@b[:,[1]]@c[1])
ang2 = np.angle(a@b[:,[0]])

In [488]:
ang3 = ang1 - ang2

In [493]:
ang3[0][0]

-3.428358599510897

In [494]:
c2 = np.exp(1j*np.array([[ang3[0][0]],[3]]))

In [495]:
np.abs(a@b@c2)

array([[23.75754371]])

In [239]:
j =2
l = 3
Ql_j = np.delete(Ql,j,axis=0)
Qlj = Ql[j,:].reshape(-1,1)
p = np.delete(wl,j).reshape(-1,1)
vect_Nr = Ql_j.T@np.exp(1j*np.angle(p))
pli_tilde = (np.angle(np.sum(np.exp(1j*np.angle(fl))*vect_Nr)) - np.angle(np.sum(np.exp(1j*np.angle(fl))*Qlj)))%np.pi
#         vl[i] = q * np.round(vli_tilde/q)

In [234]:
np.angle(np.sum(np.exp(1j*np.angle(fl))*vect_Nr))

2.0452878764042923

In [240]:
pli_tilde

2.0016016210201

In [236]:
vect_Nr.shape

(64, 1)

In [108]:
a = np.delete(fl,2)

In [109]:
b = np.delete(Ql[1,:],2)

In [117]:
c = np.sum(np.exp(1j*np.angle(a))*b)

In [118]:
c

(753.7699361747109+955.3351018636272j)

In [200]:
l = 2
i = 3

Ql_i = np.delete(Ql,i,axis=1)
Qli = Ql[:,i].reshape(-1,1)

v = np.delete(fl,i).reshape(-1,1)
vect_Nt = Ql_i@np.exp(1j*np.angle(v))
vli_tilde = (np.angle(np.sum(np.exp(1j*np.angle(wl))*vect_Nt)) - np.angle(np.sum(np.exp(1j*np.angle(wl))*Qli)))%np.pi
vli = q * np.round(vli_tilde/q)

In [185]:
n_bit = 2
q = 2*np.pi/2**n_bit

In [199]:
q

1.5707963267948966

In [198]:
vli_tilde % np.pi

1.784679891672825

In [201]:
vli

1.5707963267948966

In [194]:
4.926272545262618 - 3.14

1.786272545262618

In [168]:
x.shape

(16, 16)

In [148]:
np.angle(np.sum(np.exp(1j*np.angle(wl))*test))

2.0462362480291243